In [1]:
import psycopg2
from psycopg2 import sql, pool
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import psycopg2.extras

import asyncpg

import httpx
import asyncio
import json
from bs4 import BeautifulSoup
import datetime
import time
import os
import re
from dotenv import load_dotenv

load_dotenv()
password = os.getenv("DB_PASSWORD")
postgres_srv = os.getenv("POSTGRES_SRV")
redis_srv = os.getenv("REDIS_SRV")
openai_key = os.getenv("OPENAI_KEY")

In [24]:
db_params = {
    "database": "star_wars_data",
    "user": "postgres",
    "password": password,
    "host": postgres_srv,
    "port": "5432",
}

async def check_word(word):
  try:
    conn = await asyncpg.connect(**db_params)
    query = """
    SELECT word, similarity(word, $1)
    FROM words
    WHERE word % $1
    AND word_length > 3
    ORDER BY similarity(word, $1) DESC
    LIMIT 1;
    """
    record = await conn.fetchrow(query, word)
    await conn.close()
    if record is None:
        return None
    else:
        return record['word'], record['similarity']

  except Exception as e:
    print(f"A database error occurred: {e}")

In [28]:
await check_word("luk")

('luky', 0.5)